In [1]:
from bs4 import BeautifulSoup
from requests import get
import numpy as np
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
import itertools
import json
from shapely.geometry import Point
import matplotlib.pyplot as plt
# Standard plotly imports
import chart_studio.plotly as py
# Using plotly + cufflinks in offline mode
import cufflinks
import plotly.express as px
cufflinks.go_offline(connected=True)
import re
import time

In [2]:
def GetHtmlSoup(url, headers=()):
    try:
        response = get(url, headers=headers)
    except Exception as e:
        raise Exception(e)
        
    html_soup = BeautifulSoup(response.text, 'html.parser')
    
    return html_soup

In [3]:
def GetRetailContainer(url, name, attrs, **kwargs):
    html_soup = GetHtmlSoup(url)
    
    retail_container = html_soup.find_all(name, attrs, **kwargs)
    
    return retail_container

In [4]:
url = 'https://www.spacelist.ca/listings/b/dpz3q4u51j/dpxxegdyr8/retail-other/for-lease'
html_soup = GetHtmlSoup(url)

In [5]:
map_container = html_soup.find('div', {'id': 'cluster-map'})['data-data']
map_dict_list = json.loads(map_container)['features']
coord_dict = {f['properties']['id']: Point(f['geometry']['coordinates']) for f in map_dict_list}

In [6]:
n_pages = 70
n_ignored = 0

business_dict = {'Id': [], 'Address': [], 'Area sf': [], 'Rent/mo': [], 'Type': [], 'geometry': []}
for page in range(1, n_pages + 1):
    url = 'https://www.spacelist.ca/listings/on/toronto/office-retail-industrial-medical-multifamily-other/for-lease/page/' + str(page)
    
    retail_container = GetRetailContainer(url, 'div', {'class': 'listing-result cell shrink'})
    
    if retail_container != []:
        page_location = 0
        for retail in retail_container:
            page_location += 1
            id_value = retail.find('a', {'class': 'add-to-shortlist'})['data-listing-id']

            address_value = retail.find(
                'div',
                {'class': 'cell auto small-font dark-font flat-line-height right-cell-padding'}).text
            
            try:
                area_value = float(
                    re.findall(
                        '\d+(?:,\d+)?', 
                        retail.find('div', {'class': 'default-font dark-font flat-line-height'}).text
                    )[0].replace(',', '')
                )
            except:
                n_ignored += 1
                continue
                
            rent_text = retail.find('div', {'class': 'rent_per_month'})
            
            if rent_text is not None:
                rent_text = rent_text.text                
            else:
                sub_url = 'https://www.spacelist.ca/listings/' + id_value + '/on/toronto/for-lease/retail'
                
                sub_html_soup = GetHtmlSoup(sub_url)
                retail_container = sub_html_soup.find_all('div', {'class': 'cell small-12 large-6'})
                rent_text = retail_container[0].find('span', {'class': 'bold-font'}).text

            try:
                rent_value = float(re.search(r'\$(.*?)\/', rent_text).group(1).replace(',', ''))
            except:
                n_ignored += 1
                print(rent_text)
                continue
                
            if 'sf/yr' in rent_text:
                rent_value = (rent_value * area_value) / 12

            type_value = retail.find('div', {'class': 'cell auto about'}).contents[1].text
            
            try:
                coord_value = coord_dict[int(id_value)]
            except:
                coord_value = np.nan
            
            business_dict['Id'].append(id_value)
            business_dict['Address'].append(address_value)
            business_dict['Area sf'].append(area_value)
            business_dict['Rent/mo'].append(rent_value)
            business_dict['Type'].append(type_value)
            business_dict['geometry'].append(coord_value)
    else:
        break
        
print('Number of spaces scraped:', len(business_dict['Id']), '\n')
print('Number of spaces ignored:', n_ignored)

$309-326/mo
Contact
Contact
Contact
Contact
250 ft²
150 ft²
80 ft²
$5,418-7,563/mo
Contact
Contact
25 ft²
25 ft²
Contact
Contact
Contact
Contact
$9,053-30,750/mo
25 ft²
Contact
Contact
$1,348-31,435/mo
Contact
Contact
64 ft²
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
$7,797-15,074/mo
Contact
Contact
5,500 ft²
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
$4,380-12,135/mo
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
$6,422-21,407/mo
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Contact
Co

In [7]:
retail_sl_gdf = gpd.GeoDataFrame(pd.DataFrame(business_dict))
retail_sl_gdf.head()

,Id,Address,Area sf,Rent/mo,Type,geometry
0,299251,For Rent - 171 E Liberty St,1500.0,8300.0,Office,None
1,277036,For Rent - 32 King St E,200.0,3250.0,Office,None
2,278433,404 - 32 Britain St,165.0,3500.0,Office,None
3,191578,301 - 32 Britain St,165.0,2900.0,Office,None
4,184289,403 - 32 Britain St,260.0,6000.0,Office,None


In [8]:
retail_sl_gdf.shape

(841, 6)

In [9]:
n_unk_coord = retail_sl_gdf.loc[retail_sl_gdf['geometry'].isnull()].shape[0]
print('Number of unknown coordinates:', n_unk_coord)

Number of unknown coordinates: 727


In [10]:
retail_sl_gdf['Address'] = retail_sl_gdf.loc[:, ('Address')].apply(
    lambda s: s.replace(
        'For Rent - ', '').replace(
        'For Rent  - ', '').replace(
        'COWORKING DEDICATED - ', '').replace(
        ' COWORKING DEDICATED - ', '').replace(
        'COWORKING NOT RESERVED - ', '').replace(
        ' COWORKING NOT RESERVED - ', ''))
retail_sl_gdf['Main Address'] = retail_sl_gdf.loc[:, ('Address')].apply(
    lambda s: s.split('-')[-1].lstrip() if ('-' in s and 'HWY' not in s) else ('-'.join(s.split('-')[-2:]).lstrip() if 'HWY' in s else s))

In [11]:
na_address = np.unique(retail_sl_gdf.loc[retail_sl_gdf['geometry'].isnull()]['Main Address'])

coord_dict = {'Main Address': [], 'geometry': []}
geolocator = Nominatim(user_agent='web_scraper')
for addr in na_address:
    try:
        location = geolocator.geocode(addr + ' Toronto', timeout=10)
    except Exception as e:
        print(addr, e)
    try:
        coord_dict['Main Address'].append(addr)
        coord_dict['geometry'].append(Point(location.longitude, location.latitude))
    except Exception as e:
        print(addr, e)

geo_addr_gdf = gpd.GeoDataFrame(pd.DataFrame.from_dict(coord_dict))

retail_sl_gdf = retail_sl_gdf.merge(geo_addr_gdf, how='left', on='Main Address')
retail_sl_gdf.loc[retail_sl_gdf['geometry_x'].isnull(), 'geometry_x'] = retail_sl_gdf.loc[
    ~retail_sl_gdf['geometry_y'].isnull(), 'geometry_y']
retail_sl_gdf.drop('geometry_y', axis=1, inplace=True)
retail_sl_gdf.rename(columns={'geometry_x': 'geometry'}, inplace=True)

n_unk_coord = retail_sl_gdf.loc[retail_sl_gdf['geometry'].isnull()].shape[0]
print('Number of unknown coordinates:', n_unk_coord)

Number of unknown coordinates: 0


In [12]:
n_pages = 15
n_ignored = 0

business_dict_p2h = {'Id': [], 'Address': [], 'Area sf': [], 'Rent/mo': [], 'Type': [], 'geometry': []}
for page in range(1, n_pages + 1):
    url = 'https://www.point2homes.com/CA/Commercial-Properties-For-Lease.html?location=Toronto%2C+ON&PriceMin=0&PriceMax=1%2C000%2C000&search_mode=location&geopicker_output=853998%2C854002%2C854003%2C854008%2C854009%2C854010%2C854011%2C855493%2C855558%2C855562%2C855563&geopicker_type=nhood&geopicker_output_full=853998%7CEast+York%7COntario%2C854002%7CUptown+Toronto%7COntario%2C854003%7CYork%7COntario%2C854008%7CWest+End+Toronto%7COntario%2C854009%7CMidtown+Toronto%7COntario%2C854010%7CDowntown+Toronto%7COntario%2C854011%7CEast+End+Toronto%7COntario%2C855493%7CCaribou+Park%7CNorth+York%2C855558%7CGlen+Park%7CNorth+York%2C855562%7CLedbury+Park%7CNorth+York%2C855563%7CLawrence+Manor%7CNorth+York&page={}&SelectedView=listings&LocationGeoId=783094&viewport=-79.46408975087829%2C43.659463577335885%2C-79.35886132680602%2C43.76728718084169&zoom=13&location_changed=&ajax=1'.format(page)
    retail_container_p2h = GetRetailContainer(url, 'div', {'class': 'item-cnt clearfix'})

    if retail_container_p2h != []:
        for retail in retail_container_p2h:
            id_value = retail['id']
            
            # the lot size can be in acres, square feet, or it isn't listed
            # if it isn't listed then ignore the retail and continue the loop
            if retail.find('li', {'data-label': 'Lot Size'}) is None:
                if retail.find('li', {'data-label': 'Sqft'}) is None:
                    n_ignored += 1
                    continue
                else:
                    area_value = float(
                        re.findall(
                            '\d+(?:,\d+)?', 
                            retail.find('li', {'data-label': 'Sqft'}).text
                        )[0].replace(',', '')
                    )
            else:
                # convert acre to square feet
                area_value = float(
                    re.findall(
                        '\d+(?:.\d+)?',
                        retail.find('li', {'data-label': 'Lot Size'}).text
                    )[0]
                ) * 43560
            
            # if the rent is not given as a per month value, ignore it and continue the loop
            # the reason being that sometimes multiple spaces are listed with different rents in a
            # single listing, and the individual area and rent of each space is either not listed
            # or it's listed in paragraph form in a description field
            try:
                rent_value_month = float(
                    re.findall(
                        '\d+\,\d+', 
                        retail.find('div', {'class': 'price has-rental-term'})['data-price']
                    )[0].replace(',', '')
                )
            except:
                n_ignored += 1
                continue
            
            # it makes no sense that the rent would be $0 so ignore these ones
            if rent_value_month == 0:
                n_ignored += 1
                continue
            
            type_value = retail.find('li', {'class': 'property-type ic-proptype'}).text
            type_value = ''.join(e for e in type_value if e.isalnum())

            input_list = retail.find_all('input')
            address_value = input_list[0]['value'].split(',')[0]
            lat_value = float(input_list[1]['value'])
            lng_value = float(input_list[2]['value'])
            coord_value = Point(lng_value, lat_value)
            
            business_dict_p2h['Id'].append(id_value)
            business_dict_p2h['Address'].append(address_value)
            business_dict_p2h['Area sf'].append(area_value)
            business_dict_p2h['Rent/mo'].append(rent_value_month)
            business_dict_p2h['Type'].append(type_value)
            business_dict_p2h['geometry'].append(coord_value)
    else:
        break
        
print('Number of spaces scraped:', len(business_dict_p2h['Id']), '\n')
print('Number of spaces ignored:', n_ignored)

Number of spaces scraped: 141 

Number of spaces ignored: 94


In [13]:
retail_p2h_gdf = gpd.GeoDataFrame(pd.DataFrame(business_dict_p2h))
retail_p2h_gdf.head()

,Id,Address,Area sf,Rent/mo,Type,geometry
0,l_91891584,600 SHERBOURNE ST 507,540.00,2000.0,Office,POINT (-79.37679 43.67180)
1,l_90782040,792 Broadview ( Rear ) Ave,696.96,4000.0,Commercial,POINT (-79.35899 43.67721)
2,l_87788357,792 Broadview ( Rear ) Ave,1698.84,4500.0,Commercial,POINT (-79.35899 43.67721)
3,l_91776383,2340 Dundas St G36,566.28,1425.0,Commercial,POINT (-79.45221 43.65692)
4,l_91666075,285 EGLINTON AVE E,2260.00,7500.0,Office,POINT (-79.38806 43.70861)


In [14]:
retail_p2h_gdf.shape

(141, 6)

In [15]:
retail_gdf = gpd.GeoDataFrame(pd.concat([retail_sl_gdf, retail_p2h_gdf], ignore_index=True))
retail_gdf.head()

,Id,Address,Area sf,Rent/mo,Type,geometry,Main Address
0,299251,171 E Liberty St,1500.0,8300.0,Office,POINT (-79.41806 43.63860),171 E Liberty St
1,277036,32 King St E,200.0,3250.0,Office,POINT (-79.36448 43.65212),32 King St E
2,278433,404 - 32 Britain St,165.0,3500.0,Office,POINT (-79.37050 43.65389),32 Britain St
3,191578,301 - 32 Britain St,165.0,2900.0,Office,POINT (-79.37050 43.65389),32 Britain St
4,184289,403 - 32 Britain St,260.0,6000.0,Office,POINT (-79.37050 43.65389),32 Britain St


In [16]:
retail_gdf.shape

(982, 7)

In [17]:
# create columns for unique address IDs regardless of the source and format of the address
retail_gdf['Address ID'] = retail_gdf['Address'].apply(lambda s: ''.join(sorted(str.upper(s.replace('-', '').replace(' ', '')))))
retail_gdf['ID length'] = retail_gdf['Id'].apply(len)

In [18]:
# deal with duplicates
print(retail_gdf.shape)
retail_gdf.drop_duplicates(subset=['Address ID', 'Area sf', 'Rent/mo'], inplace=True)
print(retail_gdf.shape)
retail_gdf.drop_duplicates(subset=['Address ID', 'ID length'], inplace=True)
print(retail_gdf.shape)
retail_gdf.drop_duplicates(subset=['Address ID'], inplace=True)
print(retail_gdf.shape)

(982, 9)
(938, 9)
(910, 9)
(907, 9)


In [19]:
retail_gdf.drop(['Address ID', 'ID length'], axis=1, inplace=True)
retail_gdf.reset_index(drop=True, inplace=True)
retail_gdf.head()

,Id,Address,Area sf,Rent/mo,Type,geometry,Main Address
0,299251,171 E Liberty St,1500.0,8300.0,Office,POINT (-79.41806 43.63860),171 E Liberty St
1,277036,32 King St E,200.0,3250.0,Office,POINT (-79.36448 43.65212),32 King St E
2,278433,404 - 32 Britain St,165.0,3500.0,Office,POINT (-79.37050 43.65389),32 Britain St
3,191578,301 - 32 Britain St,165.0,2900.0,Office,POINT (-79.37050 43.65389),32 Britain St
4,184289,403 - 32 Britain St,260.0,6000.0,Office,POINT (-79.37050 43.65389),32 Britain St


Let's look at the data and see if we can find any outliers.

In [20]:
retail_gdf['Rent/sf/mo'] = retail_gdf.loc[:, ('Rent/mo')] / retail_gdf.loc[:, ('Area sf')]

In [21]:
retail_gdf.groupby('Type').agg({'Rent/sf/mo': ['count', 'mean']})

Rent/sf/mo          
                                   count      mean
Type                                              
Commercial                            61  3.259955
Industrial                            26  1.320118
Industrial / Other                     2  4.804591
Medical                                4  3.270876
Office                               639  5.663205
Office / Industrial                    4  1.584996
Office / Industrial / Other            7  2.916573
Office / Medical                       7  4.437875
Office / Other                         4  1.330906
Office / Retail                        9  2.963363
Office / Retail / Industrial          14  2.213048
Office / Retail / Land                 1  6.000000
Office / Retail / Medical              5  2.242196
Office / Retail / Multifamily          1  2.041667
Office / Retail / Other                1  2.916500
Retail                               119  3.863624
Retail / Industrial                    1  2.522936
Retail / Medical                       2  3.020619

We see that the most common types are 'Retail' and 'Office'. Furthermore, the average rent per square foot per month between the two mentioned types are very similar. Given that we are interested in using rent data for retail spaces, and the fact that the rent prices per square foot of retail and office spaces are very similar, we will only keep these two types.

In [22]:
retail_gdf = retail_gdf[['Id', 'Address', 'Area sf', 'Rent/mo', 'Rent/sf/mo', 'Type', 'geometry']]
retail_gdf.head()

,Id,Address,Area sf,Rent/mo,Rent/sf/mo,Type,geometry
0,299251,171 E Liberty St,1500.0,8300.0,5.533333,Office,POINT (-79.41806 43.63860)
1,277036,32 King St E,200.0,3250.0,16.250000,Office,POINT (-79.36448 43.65212)
2,278433,404 - 32 Britain St,165.0,3500.0,21.212121,Office,POINT (-79.37050 43.65389)
3,191578,301 - 32 Britain St,165.0,2900.0,17.575758,Office,POINT (-79.37050 43.65389)
4,184289,403 - 32 Britain St,260.0,6000.0,23.076923,Office,POINT (-79.37050 43.65389)


In [23]:
retail_gdf.to_file('rent_toronto.geojson', drive='GeoJSON', encoding='utf-8')